<a href="https://colab.research.google.com/github/sanduerhan/Licenta/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset Import

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install --upgrade numpy

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import openpyxl
from nltk.corpus import stopwords, words, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import gensim
import re, string
from string import punctuation, digits
import matplotlib.pyplot as plt
import numpy as np

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras import layers
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten ,Embedding,Input,LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

col_list = ["Column2", "Column3"]
dataset = pd.read_excel("Training.data.xlsx", usecols=col_list)
testset = pd.read_excel("Dev.xlsx", usecols=col_list)


def add_category_id(dataset):
  dataset['category_id'] = dataset['Column3'].factorize()[0]
  category_id_dataset = dataset[['Column3', 'category_id']].drop_duplicates()

  category_to_id = dict(category_id_dataset.values)
  id_to_category = dict(category_id_dataset[['category_id', 'Column3']].values)
  return dataset,category_to_id, id_to_category

# adding numerical categories
dataset, category_to_id_train, id_to_category_train = add_category_id(dataset)
testset, category_to_id_test, id_to_category_test = add_category_id(testset)

# Shuffling the datasets
dataset = dataset.sample(frac = 1)
testset = testset.sample(frac = 1)

train_x = dataset["Column2"]
train_y = dataset["category_id"]
test_x = testset["Column2"]
test_y = testset["category_id"]


In [ ]:
print(train_x[7412])

In [ ]:
# a try to keep only unique tweets (not used in paper)

train_tweets = dataset['Column2'].tolist()
test_tweets = testset['Column2'].tolist()
def keep_uniques(array, df):
    dels=[]
    for i in array:
        if array.count(i)>1:
            dels.append(i)
    dels=list(set(dels))
    for i in dels:
        df.drop( df[ df['Column2'] == i ].index, inplace=True)
    return df

dataset = keep_uniques(train_tweets, dataset)
testset = keep_uniques(test_tweets, testset)

print(len(testset))

4062


In [ ]:
# fig = plt.figure(figsize=(8,6))
colors = ['lightblue','blue','darkblue']

train_mean = dataset.groupby('Column3').Column2.count().sort_values()
print(train_mean)
test_mean = testset.groupby('Column3').Column2.count().sort_values()
X = ['Antrenare','Validare']
severe_mean = [train_mean[0], test_mean[0]]
not_mean = [train_mean[1], test_mean[1]]
moderate_mean = [train_mean[2], test_mean[2]]
X_axis = np.arange(len(X))

plt.bar(X_axis , moderate_mean, 0.2, label = 'Moderat',color ="darkblue")
plt.bar(X_axis + 0.2, not_mean, 0.2, label = 'Fără depresie',color = 'blue')
plt.bar(X_axis + 0.2*2, severe_mean, 0.2, label = 'Sever',color = 'lightblue')
  
plt.xticks(X_axis, X)
plt.ylabel("Număr de apariții")
plt.title("Tweeturi per categorie")
plt.legend()
plt.show()

In [ ]:
X = ['SMOTE', 'Ponderi de Clase', 'Nebalansat']
vec1 = [0.4224,0.4297, 0.3807]
vec2 = [0.4375,0.4505,0.4275]
vec3 = [0.4793,0.4643, 0.4612]
vec4 = [0.4776,0.4690, 0.4441]
X_axis = np.arange(3)

plt.bar(X_axis , vec4, 0.2, color ="darkblue")
plt.bar(X_axis + 0.2, vec3, 0.2, color = 'blue')
plt.bar(X_axis + 0.2*2, vec2, 0.2, color = 'lightblue')
plt.bar(X_axis + 0.2*3, vec1, 0.2, color = 'blue')
plt.xticks(X_axis, X)
plt.ylabel("Scor F1")
plt.title("Metode de balansare a datelor")
plt.legend()
plt.show()

# PreProcessing

In [ ]:
!pip install tweet-preprocessor
import preprocessor as p

def preprocess_tweets(text):
  return p.clean(text)

print(preprocess_tweets(train_x[7412]))

In [ ]:
# cleaning the text : deleting the emojis, the existing emails, the punctuation, the present digits,
# the hyperlinks and the stopwords (a,the,is, etc)

def delete_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", re.UNICODE)
    return emoji_pattern.sub(r'', text)  # no emoji


def delete_digits(text):
    text = text.lower()
    clean = text.translate(str.maketrans('', '', digits))
    return clean


def delete_punctuation(text):
    clean = text.translate(str.maketrans('', '', punctuation + '’“”'))
    return clean


stop = set(stopwords.words('english'))
stop.add('gtpoplt')
stop.add('new')
stop.add('year')
stop.add('eve')
stop.add('years')
stop.add('ti')
stop.add('ame')
stop.add('folks')
punct = list(string.punctuation)
stop.update(punct)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)


def remove_hyperlinks(text):
    for word in text:
        if re.match(r'^http', word):
            text.remove(word)
    return " ".join(text)

def remove_emails(text):
    text = text.split()
    for i in text:
        if '@' in i.strip().lower():
            text.remove(i)
    return " ".join(text)

contraction_dict = {"youre":"you are","im": "i am","wouldnt": "would not","itll": "it will","wasnt": "was not","dont": "do not","ill": "i will","isnt": "is not","cant": "cannot","arent": "are not","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


train_x = dataset["Column2"]
train_y = dataset["category_id"]
test_x = testset["Column2"]
test_y = testset["category_id"]


def denoise(train_x):
  for i in range(len(train_x)):
    train_x[i] = replace_contractions(train_x[i])
    train_x[i] = remove_emails(train_x[i])
    train_x[i] = delete_punctuation(train_x[i])
    train_x[i] = delete_emoji(train_x[i])
    train_x[i] = delete_digits(train_x[i])
    train_x[i] = remove_stopwords(train_x[i])
  return train_x

train_x = denoise(train_x)
test_x = denoise(test_x)


In [ ]:
print(test_x[449])

# Lemmatization

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
# lemmatization : crying -> cry, days -> day with pos tagging

def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemm(text):
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    sar_list_lemmatizer = [lemmatizer.lemmatize(word, get_pos(word)) for word in text]
    return " ".join(sar_list_lemmatizer)

In [ ]:
train_x = train_x.apply(lemm)
test_x = test_x.apply(lemm)

In [ ]:
print(train_x[7429])

In [ ]:
plt.figure(figsize=(12,6))
text = ' '.join(train_x[dataset['Column3']=='severe'])
wc = WordCloud(background_color='white').generate(text)
plt.imshow(wc)

# Count Vectorizer

In [8]:
# CountVectorizer reprezentation for the user tweets
def count_vec(train,test):
  vectorizer = CountVectorizer()
  x_train_cv = vectorizer.fit_transform(train)

  x_test_cv = vectorizer.transform(test)
  return x_train_cv, x_test_cv

In [9]:
x_train_cv, x_test_cv = count_vec(train_x, test_x)

# Data imbalance handling

In [12]:
sm = SMOTE(random_state = 42)
res_x, res_y = sm.fit_resample(x_train_cv, train_y)
print("After OverSampling, counts of label '2': {}".format(sum(res_y == 2)))

After OverSampling, counts of label '2': 5647


In [ ]:
os = RandomOverSampler()
res_x2, res_y2 = os.fit_resample(x_train_cv, train_y)

# Tf-Idf 

In [16]:
# Tf-idf reprezentation for the tweets
def tf_idf(train, test, y):
  vectorizer = TfidfVectorizer(max_df = 0.15,min_df = 5, ngram_range=(1,2), stop_words='english')
  train_x_tf = vectorizer.fit_transform(train)
  test_x_tf = vectorizer.transform(test)

  res_tfx, res_tfy = sm.fit_resample(train_x_tf, y)
  return train_x_tf, test_x_tf, res_tfx, res_tfy

In [17]:
train_x_tf, test_x_tf, res_tfx, res_tfy = tf_idf(train_x, test_x, train_y)

# Word2Vec

In [ ]:
sent = [row.split() for row in train_x]

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
bigram = Phraser(phrases)
sentences = bigram[sent]

In [ ]:
print(sentences)

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=5,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

In [ ]:
w2v_model.build_vocab(sentences)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
w2v_model.wv.most_similar(positive=["depression"])

In [ ]:
model,word_index,pad_rev_train = word2(train_x)

In [ ]:
model2,word_index2,pad_rev_test = word2(test_x)

In [35]:
def get_weight_matrix(model, vocab):
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, 300))
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [ ]:
embedding_vectors = get_weight_matrix(model, word_index)

In [36]:
def w2v_embedding(train_text):

  lst_corpus = []
  for string in train_text:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) 
                for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)

  bigrams_detector = gensim.models.phrases.Phrases(lst_corpus, 
                  delimiter=" ".encode(), min_count=30)
  bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)
  trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[lst_corpus], 
              delimiter=" ".encode(), min_count=30)
  trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

  nlp = Word2Vec(lst_corpus, size=300,   
            window=8, min_count=2, sg=1, iter=30)
  return nlp, lst_corpus, bigrams_detector, trigrams_detector

In [37]:
def feature_engineering(lst_corpus, train_text):
  tokenizer = Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
  tokenizer.fit_on_texts(lst_corpus)
  dic_vocabulary = tokenizer.word_index

  lst_corpus = []
  for string in train_text:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) 
                for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)
  lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)

  X_train = pad_sequences(lst_text2seq, 
                      maxlen=30, padding="post", truncating="post")
  return tokenizer, dic_vocabulary, X_train

In [38]:
def make_embedding_matrix(dic_vocabulary, nlp):
  embeddings = np.zeros((len(dic_vocabulary)+1, 300))
  for word,idx in dic_vocabulary.items():
      try:
          embeddings[idx] =  nlp[word]
      except:
          pass
  return embeddings

In [39]:
nlp, lst_corpus, bigrams_detector, trigrams_detector = w2v_embedding(train_x)

In [ ]:
print(nlp.wv.most_similar(positive="depression"))

In [40]:
tokenizer, dic_vocabulary, X_train = feature_engineering(lst_corpus,train_x)

In [ ]:
print(X_train.shape)

In [41]:
embeddings_train = make_embedding_matrix(dic_vocabulary, nlp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [44]:
def test_handling(test_text,bigrams_detector, trigrams_detector, tokenizer):

  lst_corpus = []
  for string in test_text:
      lst_words = string.split()
      lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, 
                  len(lst_words), 1)]
      lst_corpus.append(lst_grams)

  lst_corpus = list(bigrams_detector[lst_corpus])
  lst_corpus = list(trigrams_detector[lst_corpus])

  lst_text2seq = tokenizer.texts_to_sequences(lst_corpus)

  X_test = pad_sequences(lst_text2seq, maxlen=30,
              padding="post", truncating="post")
  return X_test

In [42]:
nlp2, lst_corpus2, bigrams_detector2, trigrams_detector2 = w2v_embedding(test_x)

In [45]:
x_test = test_handling(test_x, bigrams_detector2, trigrams_detector2, tokenizer)

In [ ]:
print(x_test.shape)

# Naive Bayes

In [10]:
def naive_bayes(train_x, train_y, test_x, test_y):
  naive_bayes_classifier = MultinomialNB()
  naive_bayes_classifier.fit(train_x, train_y)
  pred_y = naive_bayes_classifier.predict(test_x)

  score1 = metrics.accuracy_score(test_y, pred_y)
  print("Accuracy " + str(score1))
  print(metrics.classification_report(test_y, pred_y, digits = 4))

In [13]:
naive_bayes(res_x, res_y, x_test_cv, test_y)

Accuracy 0.5272192251103482
              precision    recall  f1-score   support

           0     0.5720    0.8353    0.6790      2198
           1     0.6351    0.1498    0.2424      1522
           2     0.1690    0.2402    0.1984       358

    accuracy                         0.5272      4078
   macro avg     0.4587    0.4084    0.3733      4078
weighted avg     0.5601    0.5272    0.4739      4078



In [14]:
naive_bayes(x_train_cv, train_y, x_test_cv, test_y)

Accuracy 0.5259931338891614
              precision    recall  f1-score   support

           0     0.5739    0.8130    0.6728      2198
           1     0.5497    0.1380    0.2206      1522
           2     0.2543    0.4134    0.3149       358

    accuracy                         0.5260      4078
   macro avg     0.4593    0.4548    0.4028      4078
weighted avg     0.5368    0.5260    0.4726      4078



In [18]:
naive_bayes(res_tfx, res_tfy, test_x_tf, test_y)

Accuracy 0.280039234919078
              precision    recall  f1-score   support

           0     0.5653    0.2798    0.3743      2198
           1     0.6378    0.1353    0.2233      1522
           2     0.1204    0.8966    0.2122       358

    accuracy                         0.2800      4078
   macro avg     0.4411    0.4373    0.2700      4078
weighted avg     0.5533    0.2800    0.3037      4078



In [22]:
naive_bayes(train_x_tf, train_y, test_x_tf, test_y)

Accuracy 0.5399705738106915
              precision    recall  f1-score   support

           0     0.5395    1.0000    0.7009      2198
           1     1.0000    0.0020    0.0039      1522
           2     1.0000    0.0028    0.0056       358

    accuracy                         0.5400      4078
   macro avg     0.8465    0.3349    0.2368      4078
weighted avg     0.7518    0.5400    0.3797      4078



# AdaBoost

In [23]:

def adaboost(train_x, train_y, test_x, test_y):
  ada = AdaBoostClassifier()
  boost = ada.fit(train_x, train_y)
  pred_y = boost.predict(test_x)
  print("AdaBoost Classifier Model Accuracy:", accuracy_score(test_y, pred_y))
  print(metrics.classification_report(test_y, pred_y))

In [24]:
adaboost(res_x, res_y, x_test_cv, test_y)

AdaBoost Classifier Model Accuracy: 0.4492398234428642
              precision    recall  f1-score   support

           0       0.62      0.48      0.54      2198
           1       0.54      0.41      0.46      1522
           2       0.12      0.41      0.19       358

    accuracy                           0.45      4078
   macro avg       0.43      0.43      0.40      4078
weighted avg       0.55      0.45      0.48      4078



In [25]:
adaboost(x_train_cv, train_y, x_test_cv, test_y)

AdaBoost Classifier Model Accuracy: 0.5223148602256008
              precision    recall  f1-score   support

           0       0.54      0.90      0.68      2198
           1       0.43      0.05      0.09      1522
           2       0.32      0.25      0.28       358

    accuracy                           0.52      4078
   macro avg       0.43      0.40      0.35      4078
weighted avg       0.48      0.52      0.42      4078



In [26]:
adaboost(res_tfx, res_tfy, test_x_tf, test_y)

AdaBoost Classifier Model Accuracy: 0.4521824423737126
              precision    recall  f1-score   support

           0       0.57      0.38      0.46      2198
           1       0.48      0.55      0.51      1522
           2       0.20      0.48      0.28       358

    accuracy                           0.45      4078
   macro avg       0.42      0.47      0.42      4078
weighted avg       0.50      0.45      0.46      4078



In [27]:
adaboost(train_x_tf, train_y, test_x_tf, test_y)

AdaBoost Classifier Model Accuracy: 0.5122609122118685
              precision    recall  f1-score   support

           0       0.54      0.86      0.67      2198
           1       0.53      0.04      0.08      1522
           2       0.27      0.35      0.31       358

    accuracy                           0.51      4078
   macro avg       0.45      0.42      0.35      4078
weighted avg       0.51      0.51      0.42      4078



# Regression

In [20]:
class_weight = {0: 1.,
                1: 3.,
                2: 4.}

def logistic_regression(train_x, train_y, test_x, test_y):
  model = LogisticRegression(multi_class='multinomial', solver='newton-cg')

  model = model.fit(train_x, train_y)
  y_pred = model.predict(test_x)

  print("Regression Classifier Model Accuracy:", accuracy_score(test_y, y_pred))
  print(metrics.classification_report(test_y, y_pred, digits = 4))

  conf_mat = confusion_matrix(test_y, y_pred)
  print(conf_mat)


In [ ]:
logistic_regression(res_x, res_y, x_test_cv, test_y)

In [ ]:
logistic_regression(x_train_cv, train_y, x_test_cv, test_y)

In [ ]:
logistic_regression(res_tfx, res_tfy, test_x_tf, test_y)

In [ ]:
logistic_regression(train_x_tf, train_y, test_x_tf, test_y)

# SVM

In [30]:

class_weight = {0: 1.,
                1: 3.,
                2: 6.}

def svm(train_x, train_y, test_x, test_y):
  model = LinearSVC()
  model = model.fit(train_x_tf, train_y)
  y_pred = model.predict(test_x_tf)
  
  print("Svc Classifier Model Accuracy:", accuracy_score(test_y, y_pred))
  print(metrics.classification_report(test_y, y_pred, digits = 4))

  conf_mat = confusion_matrix(test_y, y_pred)
  print(conf_mat)


In [ ]:
svm(res_x, res_y, x_test_cv, test_y)

In [ ]:
svm(x_train_cv, train_y, x_test_cv, test_y)

In [ ]:
svm(res_tfx, res_tfy, test_x_tf, test_y)

In [31]:
svm(train_x_tf, train_y, test_x_tf, test_y)

Svc Classifier Model Accuracy: 0.5465914664051006
              precision    recall  f1-score   support

           0     0.5866    0.7552    0.6603      2198
           1     0.4787    0.2654    0.3415      1522
           2     0.4084    0.4609    0.4331       358

    accuracy                         0.5466      4078
   macro avg     0.4912    0.4939    0.4783      4078
weighted avg     0.5307    0.5466    0.5214      4078

[[1660  388  150]
 [1029  404   89]
 [ 141   52  165]]


# F1 metric

In [32]:

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [33]:
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

# Neural Network

In [ ]:
class_weight = {0: 1.,
                1: 3.,
                2: 5.}
# def split(pad_rev,train_y):
#   Y=keras.utils.to_categorical(train_y)  # one hot target as required by NN.
#   x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.05,random_state=42)
#   return x_train, x_test,y_train,y_test
def network(embeddings):
 ## input
  x_in = layers.Input(shape=(30,))
  ## embedding
  x = layers.Embedding(input_dim=embeddings.shape[0],  
                      output_dim=embeddings.shape[1], 
                      weights=[embeddings],
                      input_length=30, trainable=False)(x_in)
  ## 2 layers of bidirectional lstm
  x = layers.Bidirectional(layers.LSTM(units=30, dropout=0.2, 
                          return_sequences=True))(x)
  x = layers.Bidirectional(layers.LSTM(units=30, dropout=0.2))(x)
  ## final dense layers
  x = layers.Dense(64, activation='elu')(x)
  y_out = layers.Dense(3, activation='softmax')(x)
  ## compile
  model = Model(x_in, y_out)
  model.compile(loss=f1_loss,
                optimizer='adam', metrics=[f1])
  return model
Y=keras.utils.to_categorical(train_y)
Y2=keras.utils.to_categorical(test_y)
# x_train,x_test,y_train,y_test = split(X_train,train_y)
model = network(embeddings_train)
# x_valid, x_x, y_valid, y_y = split(x_test,test_y)
model.fit(X_train, Y, epochs=10, batch_size=64, validation_data=(x_test,Y2))


In [ ]:
model.summary()

In [ ]:
pred_y = model.predict(x_test)
pred_y = np.argmax(pred_y, axis=1)
# y_test=np.argmax(y_test, axis=1)
print(metrics.classification_report(test_y, pred_y, digits = 4))

conf_mat = confusion_matrix(test_y, pred_y)
print(conf_mat)

# BERT

In [ ]:
!pip install transformers

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

In [50]:
from keras.initializers import GlorotNormal
from tensorflow.keras.callbacks import EarlyStopping

In [51]:
from transformers import RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
text_train = train_x.astype(str).tolist()
text_test = test_x.astype(str).tolist()

In [ ]:
s = tokenizer.tokenize(train_x)
embt = tokenizer.encode(s)

In [ ]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

In [ ]:
def sentence(texts):
  model = SentenceTransformer("all-mpnet-base-v2")
  embeddings = model.encode(texts)

  return embeddings

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")

In [53]:
TRANSFORMER_BATCH=128

def count_embedd (df):
    idx_chunk=list(df.columns).index('Column2')
    embedd_lst = []
    for index in range (0, len(df), TRANSFORMER_BATCH):
        embedds = model.encode(df.iloc[index:index+TRANSFORMER_BATCH, idx_chunk].values, show_progress_bar=False)
        embedd_lst.append(embedds)
    return np.concatenate(embedd_lst)

In [54]:
train_emb = count_embedd(dataset)

In [55]:
test_emb = count_embedd(testset)

In [56]:
X_train = np.array(train_emb)
X_test = np.array(test_emb)

In [57]:
y_train=keras.utils.to_categorical(train_y)

In [ ]:
KERAS_VALIDATION_SPLIT=0.00
KERAS_EPOCHS=10
KERAS_BATCH_SIZE=128

class_weight = {0: 1.,
                1: 3.,
                2: 6.}

# Create and train Keras model
n_features=X_train.shape[1]
n_labels = y_train.shape[1]

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(2048, input_dim=n_features),
  tf.keras.layers.LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64),
  tf.keras.layers.LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3, activation='softmax')
])

LR=5e-4
adam = keras.optimizers.Adam(learning_rate=LR, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=adam, 
              loss="categorical_crossentropy",
              metrics=[f1])

model.fit(X_train, y_train, epochs=KERAS_EPOCHS, batch_size=KERAS_BATCH_SIZE, validation_split=KERAS_VALIDATION_SPLIT)

In [ ]:
model.summary()


In [ ]:
y_preds = model.predict(X_test)
predicted = np.argmax(y_preds,axis=1) 
accuracy = metrics.accuracy_score(test_y, predicted)
print("Accuracy:",  round(accuracy,4))
print(metrics.classification_report(test_y, predicted,digits=4))

conf_mat = confusion_matrix(test_y, predicted)
print(conf_mat)